In [2]:
 torch.cuda.get_device_capability

NameError: name 'torch' is not defined

In [8]:
import torch
#from transformers import VivitModel, VivitImageProcessor
import imageio
import cv2
import numpy as np
import os
import time

In [5]:
# Load the ViViT Vision Model from Hugging Face
model_name = "google/vivit-b-16x2-kinetics400"
vivit_model = transformers.VivitModel.from_pretrained(model_name)
image_processor = transformers.VivitImageProcessor.from_pretrained(model_name)
vivit_model.eval()  # Set the model to evaluation mode
print("ViViT model loaded successfully.")

ModuleNotFoundError: Could not import module 'VivitModel'. Are this object's requirements defined correctly?

In [4]:
# Preprocess Videos for ViViT
def preprocess_video(video_path, image_processor, num_frames=32):
    video_reader = imageio.get_reader(video_path, "ffmpeg")
    frames = []
    for i, frame in enumerate(video_reader):
        if len(frames) >= num_frames:
            break
        frames.append(frame)
    video_reader.close()
    # Repeat frames until num_frames is reached
    while len(frames) < num_frames:
        frames.extend(frames[:num_frames - len(frames)])
    # Use the feature extractor for preprocessing
    inputs = image_processor(frames, return_tensors="pt")
    return inputs


In [5]:
# Encode Videos Using ViViT
def encode_video(vivit_model, preprocessed_video):
    # Pass through the ViViT model
    with torch.no_grad():
        outputs = vivit_model(**preprocessed_video)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Example: Use mean pooling
    return embeddings



In [6]:

def store_embeddings_in_faais(index, video_path, embeddings):
    video_name = os.path.basename(video_path)
    metadata = {"video_name": video_name}
    index.add(embeddings.numpy())
    print(f"Stored embeddings for {video_name} in FAISS.")




In [ ]:
locals()

In [8]:
# Define the folder containing the videos
video_folder = "./data/jefit/mp4"
faiss_index = None
last_save_time = time.time()
save_interval=300
# Loop through all video files in the folder
for video_file in os.listdir(video_folder):
    if video_file.endswith(".mp4"):  # Check if the file is an MP4 video
        video_path = os.path.join(video_folder, video_file)
        
        # Preprocess the video
        preprocessed_video = preprocess_video(video_path, image_processor)
        
        # Encode the video
        video_embeddings = encode_video(vivit_model, preprocessed_video)
        # Store Encoded Features in FAAIS
        dimension = video_embeddings.shape[1]
        if(faiss_index is None):
            # Initialize the FAISS index
            # Use L2 distance for similarity search
            faiss_index = faiss.IndexFlatL2(dimension)  # L2 distance index
        
        # Store the embeddings in the FAISS index
        store_embeddings_in_faais(faiss_index, video_path, video_embeddings)
        if(time.time() - last_save_time >= save_interval):
            # Save the FAISS index to disk
            faiss.write_index(faiss_index, "faiss_index.index")
            last_save_time = time.time()

/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(value)


Stored embeddings for 578.mp4 in FAISS.
Stored embeddings for 544.mp4 in FAISS.
Stored embeddings for 222.mp4 in FAISS.
Stored embeddings for 236.mp4 in FAISS.
Stored embeddings for 550.mp4 in FAISS.
Stored embeddings for 1013.mp4 in FAISS.
Stored embeddings for 587.mp4 in FAISS.
Stored embeddings for 593.mp4 in FAISS.
Stored embeddings for 1007.mp4 in FAISS.
Stored embeddings for 746.mp4 in FAISS.
Stored embeddings for 752.mp4 in FAISS.
Stored embeddings for 961.mp4 in FAISS.
Stored embeddings for 1239.mp4 in FAISS.
Stored embeddings for 975.mp4 in FAISS.
Stored embeddings for 785.mp4 in FAISS.
Stored embeddings for 1211.mp4 in FAISS.
Stored embeddings for 1205.mp4 in FAISS.
Stored embeddings for 791.mp4 in FAISS.
Stored embeddings for 949.mp4 in FAISS.
Stored embeddings for 168.mp4 in FAISS.
Stored embeddings for 96.mp4 in FAISS.
Stored embeddings for 82.mp4 in FAISS.
Stored embeddings for 154.mp4 in FAISS.
Stored embeddings for 632.mp4 in FAISS.
Stored embeddings for 626.mp4 in FAIS

In [9]:
faiss_index = faiss.read_index("faiss_index.index")

In [48]:
preprocessed_video = preprocess_video('/Users/kanantharaman/Downloads/fitclass_converted_-cAGR3GiZIU_122_377.mp4', image_processor)        # Encode the video
video_embeddings = encode_video(vivit_model, preprocessed_video)

In [63]:
# Perform a FAISS search for the top 5 nearest neighbors
D, I = faiss_index.search(video_embeddings.numpy(), 3)
print("Distances:", D)
print("Indices:", I)

Distances: [[286.14584 347.96307 357.20306]]
Indices: [[795 201 341]]


In [19]:
os.listdir('./data/jefit/mp4')[90]

'237.mp4'

In [40]:
exercises = {}
with open('./exercise_links.txt', 'r') as f:
    for line in f.readlines():
        id = line.split('/')[-2]
        name = line.split('/')[-1]
        exercises[id] = name.strip()

In [45]:
video_folder = "./data/jefit/mp4"
exercise_list = []
for video_file in os.listdir(video_folder):
    id = video_file.split('.')[0]
    name = exercises[id]
    exercise_list.append((id, name))
    

In [46]:
exercise_list

[('578', 'rotational-step-up'),
 ('544', 'stability-ball-back-stretch'),
 ('222', 'dumbbell-decline-tricep-extension'),
 ('236', 'dumbbell-tricep-extension-supine'),
 ('550', 'stability-ball-half-moon-stretch'),
 ('1013', 'cable-seated-bicep-curl'),
 ('587', 'stability-ball-incline-ab-crunch'),
 ('593', 'stability-ball-dumbbell-sit-up'),
 ('1007', 'dumbbell-alternating-preacher-curl-hammer'),
 ('746', 'hip-adduction'),
 ('752', 'barbell-squat-to-upright-row'),
 ('961', 'peroneal-stretch'),
 ('1239', 'smith-machine-single-leg-calf-raise'),
 ('975', 'decline-bench-leg-raise-with-hip-thrust'),
 ('785', 'cable-deadlift'),
 ('1211', 'stability-ball-weighted-wall-squat'),
 ('1205', 'stability-ball-wall-squat'),
 ('791', 'bench-oblique-crunch'),
 ('949', 'standing-hamstring-and-calf-stretch'),
 ('168', 'barbell-decline-pullover-wide-grip'),
 ('96', 'cable-dual-overhead-curl'),
 ('82', 'chin-up-with-knee-raise'),
 ('154', 'dumbbell-one-arm-tricep-kickback'),
 ('632', 'stability-ball-plank-with

In [64]:
for i in I[0]:
    print(exercise_list[i])

('672', 'straight-leg-sit-up')
('967', 'abdominal-pendulum')
('1361', 'side-lying-hip-abduction')


In [54]:
from dotenv import load_dotenv
import pyarrow.fs as fs

load_dotenv()
access_key = os.getenv('access_key')
secret_key = os.getenv('secret_key')
region = os.getenv('region')
namespace = os.getenv('namespace')
s3 = fs.S3FileSystem(region=region, access_key=access_key, secret_key=secret_key, endpoint_override=f'https://{namespace}.compat.objectstorage.{region}.oraclecloud.com',)
input_path = 'yt-vtt/fitclass/'
files = s3.get_file_info(fs.FileSelector(input_path))
json_files = [file.path for file in files if file.path.endswith('.json')]
mp4_files = [file.path for file in files if file.path.endswith('.mp4')]


In [62]:
preprocessed_video = preprocess_video(f'./data/{video_file}', image_processor)
video_embeddings = encode_video(vivit_model, preprocessed_video)

In [70]:
for video_file in mp4_files:
    if(os.path.exists(f'./data/{video_file}')):
        print(f'File {video_file} already exists.')
    else:
     with s3.open_input_file(video_file) as video:
        with open(f'./data/{video_file}', 'wb') as local_file:
            local_file.write(video.read())
        
        preprocessed_video = preprocess_video(f'./data/{video_file}', image_processor)
        video_embeddings = encode_video(vivit_model, preprocessed_video)
        D, I = faiss_index.search(video_embeddings.numpy(), 3)
        for i in I[0]:
            print(video_file,exercise_list[i])


File yt-vtt/fitclass/converted_-cAGR3GiZIU_10572_11595.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_11847_12358.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_122_377.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_12774_12997.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_13455_14045.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_14274_14546.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_14724_15235.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_15472_15618.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_15724_15855.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_15978_16489.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_17780_18803.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_18861_19052.mp4 already exists.
File yt-vtt/fitclass/converted_-cAGR3GiZIU_1924_2947.mp4 already exists.
File yt-vtt/fitclass/converted_

KeyboardInterrupt: 

In [74]:
video_file = "./data/yt-vtt/fitclass/converted_1ADarF8TSxE_15221_16240.mp4"
preprocessed_video = preprocess_video(video_file, image_processor)
video_embeddings = encode_video(vivit_model, preprocessed_video)
D, I = faiss_index.search(video_embeddings.numpy(), 3)
for i in I[0]:
            print(video_file,exercise_list[i])

./data/yt-vtt/fitclass/converted_1ADarF8TSxE_15221_16240.mp4 ('1366', 'dumbbell-squat-to-shoulder-press')
./data/yt-vtt/fitclass/converted_1ADarF8TSxE_15221_16240.mp4 ('960', 'wall-calf-stretch-')
./data/yt-vtt/fitclass/converted_1ADarF8TSxE_15221_16240.mp4 ('474', 'dumbbell-bench-squat-')


In [4]:
import  torchvision.transforms.v2 
class ToPILImage:
    def __init__(self):
        self.to_pil = torchvision.transforms.ToPILImage()
    def __call__(self, frames):
        print(len(frames))
        frames = [self.to_pil(frame) for frame in frames]
        return frames
class Resize:
    def __init__(self, h,w):
        self.h = h
        self.w = w  
        self.resize = torchvision.transforms.v2.Resize((h,w))
    def __call__(self, frames):
        frames = [self.resize(frame) for frame in frames]
        return frames

class ToTensor:
    def __init__(self):
        self.to_tensor = torchvision.transforms.v2.ToTensor()
    def __call__(self, frames):
        frames = [self.to_tensor(frame) for frame in frames]
        return frames
    
class SampleFrames:
    def __init__(self, num_frames=32):
        self.num_frames = num_frames

    def __call__(self, frames):
        frames = torch.stack(frames)
        if frames.shape[0] < self.num_frames:
            # Repeat frames until the desired number is reached
            repeated_frames = frames.repeat((self.num_frames // frames.shape[0], 1, 1, 1))
            remainder = self.num_frames % frames.shape[0]
            if remainder > 0:
                repeated_frames = torch.cat((repeated_frames, frames[:remainder]), dim=0)
            return repeated_frames
        else:
            # Use UniformTemporalSubsample to subsample frames
            print("Subsampling frames")
            subsample = torchvision.transforms.v2.UniformTemporalSubsample(self.num_frames)
            return subsample(frames.unsqueeze(0)).squeeze(0)

In [5]:
from torchvision.datasets import DatasetFolder
from torchvision.transforms.v2 import Compose

# Define a custom loader for GIF files
def gif_loader(path):
    frames = []
    gif_frames =imageio.mimread(path, memtest=False)
    for frame in gif_frames:
                        # Check if the frame has 4 channels (RGBA)
                        if frame.shape[-1] == 4:
                            # Convert RGBA to RGB
                            frame = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)
                        frames.append(frame)
    return frames#np.array(frames)#.transpose(0, 3, 1, 2)  # Convert to (frames, channels, height, width)
# Define a transform to apply to each frame of the GIF
transform = Compose([
    
    #
    ToPILImage(),  # Convert frames to PIL images
    Resize(224, 224),  # Resize frames to 224x224
    ToTensor(),
    SampleFrames(),
    #ToTensor(),          # Convert frames to PyTorch tensors
    
])

# Create a custom dataset using DatasetFolder
train_dataset = DatasetFolder(
    root="./data/kinetics1/train",
    loader=gif_loader,
    extensions=("gif",),
    transform=transform  # Use the existing transform variable
)


test_dataset = DatasetFolder(
    root="./data/kinetics1/test",
    loader=gif_loader,
    extensions=("gif",),
    transform=transform  # Use the existing transform variable
)

print(f"Custom dataset created with {len(train_dataset)} samples.")
print(f"Test dataset created with {len(test_dataset)} samples.")

Custom dataset created with 2321 samples.
Test dataset created with 589 samples.


/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [10]:
train_dataset.classes

['bench_press',
 'calf_raise',
 'cardio',
 'crunch',
 'curl_(biceps)',
 'deadlift',
 'extension_(triceps)',
 'fly',
 'hip_thrust',
 'leg_raise',
 'lunge',
 'pull-up__chin-up',
 'push-up',
 'row',
 'shoulder_press',
 'shoulder_raise',
 'squat',
 'stretch__mobility',
 'twist__rotation']

In [9]:
from torch.utils.data import DataLoader

# Create a DataLoader for the custom_dataset
batch_size = 2  # Define the batch size
shuffle = True   # Shuffle the data
num_workers = 1 # Number of subprocesses for data loading

data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

# Example: Iterate through the DataLoader
for batch_idx, (data, labels) in enumerate(data_loader):
    print(f"Batch {batch_idx}:")
    print(f"Data shape: {len(data)} {data[0].shape}")  # Print shape of the first item in the batch
    print(f"Labels: {labels}")
    break  # Process only the first batch for demonstration

25


TypeError: expected np.ndarray (got numpy.ndarray)

In [17]:
from collections import defaultdict

# Path to the train directory
train_dir = "./data/kinetics1/train"
test_dir = "./data/kinetics1/test"

# Dictionary to store the count of files by directory
file_count_by_dir = defaultdict(int)

# Walk through the train directory
for root, dirs, files in os.walk(train_dir):
    if(len(files) == 0):
        print(f"Empty directory: {root}")
    for file in files:
        file_count_by_dir[os.path.basename(root)] += 1

# Print the number of files by directory
for directory, count in file_count_by_dir.items():
        print(f"Directory: {directory}, Number of files: {count}")
        

Empty directory: ./data/kinetics1/train
Directory: calf_raise, Number of files: 96
Directory: deadlift, Number of files: 78
Directory: squat, Number of files: 194
Directory: push-up, Number of files: 95
Directory: fly, Number of files: 96
Directory: stretch__mobility, Number of files: 205
Directory: leg_raise, Number of files: 74
Directory: cardio, Number of files: 91
Directory: row, Number of files: 208
Directory: shoulder_raise, Number of files: 111
Directory: crunch, Number of files: 97
Directory: lunge, Number of files: 77
Directory: twist__rotation, Number of files: 131
Directory: curl_(biceps), Number of files: 237
Directory: extension_(triceps), Number of files: 154
Directory: bench_press, Number of files: 128
Directory: shoulder_press, Number of files: 92
Directory: pull-up__chin-up, Number of files: 80
Directory: hip_thrust, Number of files: 76


In [21]:
train_dir = "./data/kinetics2/test"

# Walk through the train directory
for root, dirs, files in os.walk(train_dir):
    if(len(files) == 0):
        print(f"Empty directory: {root}")
    for file in files:
        try:
            # Load the GIF frames using the custom loader
            gif_frames = gif_loader(os.path.join(root, file))
            #print(root, file, len(gif_frames))
        except Exception as e:
            print(f"Error loading {file} in {root}: {e}")
            continue


Empty directory: ./data/kinetics2/test
Error loading lvsmsplitsquat.gif in ./data/kinetics2/test/squat: image file is truncated (205 bytes not processed)


In [16]:
file_to_remove = './data/kinetics1/train/squat/lvsmsplitsquat.gif'
if os.path.exists(file_to_remove):
    os.remove(file_to_remove)
    print(f"File {file_to_remove} has been removed.")
else:
    print(f"File {file_to_remove} does not exist.")

File ./data/kinetics1/train/squat/lvsmsplitsquat.gif has been removed.


In [ ]:
from torchvision.datasets import DatasetFolder
import os
import random

class CustomDatasetFolder(DatasetFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir()]
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        
        # Ensure each class has at most 75 samples
        for cls_name in classes:
            cls_path = os.path.join(directory, cls_name)
            gif_files = [f for f in os.listdir(cls_path) if f.endswith('.gif')]
            selected_gif_files = random.sample(gif_files, min(75, len(gif_files)))
            
            # Remove unselected files
            for gif_file in gif_files:
                if gif_file not in selected_gif_files:
                    os.remove(os.path.join(cls_path, gif_file))
        
        return classes, class_to_idx
    def make_dataset(self, directory, class_to_idx, extensions=None, is_valid_file=None):
        instances = []
        for target_class in sorted(class_to_idx.keys()):
            class_idx = class_to_idx[target_class]
            target_dir = os.path.join(directory, target_class)
            if not os.path.isdir(target_dir):
                continue
            gif_files = [f for f in os.listdir(target_dir) if f.endswith('.gif')]
            selected_gif_files = random.sample(gif_files, min(75, len(gif_files)))
            for gif_file in selected_gif_files:
                path = os.path.join(target_dir, gif_file)
                item = (path, class_idx)
                instances.append(item)
        return instances

# Example usage
custom_train_dataset = CustomDatasetFolder(
    root="./data/kinetics1/train",
    loader=gif_loader,
    extensions=("gif",),
    transform=transform
)

print(f"Custom train dataset created with {len(custom_train_dataset)} samples.")